In [1]:
import numpy as np
import pandas as pd
import json
import re
import csv
import ast

In [2]:
csop_data_csv = pd.read_csv('../raw_data/RAT_chat.csv')
dat_data_csv = pd.read_csv('../raw_data/DAT_chat.csv')

In [4]:
# split up the easy v. hard
csop_easy = csop_data_csv[["id", "batch_num", "vis_img", "int_verb", "ort_img", "rep_man", "soc_pers", "team_size", "score_easy", "time_easy", "eff_easy", "chat_easy"]]
csop_easy = csop_easy.rename(columns={"score_easy": "score", "time_easy": "duration", "eff_easy": "efficiency", "chat_easy": "message"})
csop_easy["difficulty"] = "Easy [Corresponds to 'Hard' in PNAS]"
csop_easy["id"] = csop_easy["id"] + "_easy"

csop_hard = csop_data_csv[["id", "batch_num", "vis_img", "int_verb", "ort_img", "rep_man", "soc_pers", "team_size", "score_hard", "time_hard", "eff_hard", "chat_hard"]]
csop_hard = csop_hard.rename(columns={"score_hard": "score", "time_hard": "duration", "eff_hard": "efficiency", "chat_hard": "message"})
csop_hard["difficulty"] = "Hard [Corresponds to 'Super Hard' in PNAS]"
csop_hard["id"] = csop_hard["id"] + "_hard"

In [5]:
csop_easy.head()

,id,batch_num,vis_img,int_verb,ort_img,rep_man,soc_pers,team_size,score,duration,efficiency,message,difficulty
0,rHPaiuXkM3Ss4rEsW_easy,1,High,High,High,High,High,5,586.0,107.0,5.476636,"{""text"":""Can we get D to the highest score?"",""...",Easy [Corresponds to 'Hard' in PNAS]
1,dArSAcrzmb9bR6Pug_easy,1,High,High,High,High,High,5,559.0,406.0,1.376847,"{""text"":""looks good"",""playerId"":""6QvNn8wdCyviK...",Easy [Corresponds to 'Hard' in PNAS]
2,sbtKHuLChG8ge734n_easy,1,High,High,High,High,Low,8,484.0,123.0,3.934959,NaN,Easy [Corresponds to 'Hard' in PNAS]
3,guq9kzMuDCpB4hRhK_easy,1,Mixed,High,High,High,Low,4,515.0,152.0,3.388158,"{""text"":""wow"",""playerId"":""F95Kc45STrQWhYNDa"",""...",Easy [Corresponds to 'Hard' in PNAS]
4,hSNqR8mxGD2jv3WeZ_easy,1,High,Low,High,High,High,1,653.0,457.0,1.428884,NaN,Easy [Corresponds to 'Hard' in PNAS]


In [6]:
csop_data_csv = pd.concat([csop_easy, csop_hard])
csop_data_csv.head()

,id,batch_num,vis_img,int_verb,ort_img,rep_man,soc_pers,team_size,score,duration,efficiency,message,difficulty
0,rHPaiuXkM3Ss4rEsW_easy,1,High,High,High,High,High,5,586.0,107.0,5.476636,"{""text"":""Can we get D to the highest score?"",""...",Easy [Corresponds to 'Hard' in PNAS]
1,dArSAcrzmb9bR6Pug_easy,1,High,High,High,High,High,5,559.0,406.0,1.376847,"{""text"":""looks good"",""playerId"":""6QvNn8wdCyviK...",Easy [Corresponds to 'Hard' in PNAS]
2,sbtKHuLChG8ge734n_easy,1,High,High,High,High,Low,8,484.0,123.0,3.934959,NaN,Easy [Corresponds to 'Hard' in PNAS]
3,guq9kzMuDCpB4hRhK_easy,1,Mixed,High,High,High,Low,4,515.0,152.0,3.388158,"{""text"":""wow"",""playerId"":""F95Kc45STrQWhYNDa"",""...",Easy [Corresponds to 'Hard' in PNAS]
4,hSNqR8mxGD2jv3WeZ_easy,1,High,Low,High,High,High,1,653.0,457.0,1.428884,NaN,Easy [Corresponds to 'Hard' in PNAS]


In [7]:
# write the CSOP II data
with open('../raw_data/csopII_conversations_withblanks.csv', 'w', newline='') as csvfile:
    # start writing the header of the CSV
    fieldnames = ["conversation_num", "batch_num","vis_img", "int_verb", "ort_img", "rep_man", "soc_pers", "team_size", "difficulty", "score", "duration", "efficiency", "speaker_nickname", "message", "timestamp"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for index, row in csop_data_csv.iterrows():
        try:
            chat = "[" + row['message'] + "]" # convert messages to a list of dict objects
            chat_list = eval(chat)
            # filter such that we only look at people who actually said something
            for chat in chat_list:
                writer.writerow({'conversation_num': row['id'], 'batch_num': row['batch_num'], 'vis_img': row['vis_img'], 'int_verb': row['int_verb'], 'ort_img': row['ort_img'], 'rep_man': row['rep_man'], 'soc_pers': row['soc_pers'], 'team_size': row['team_size'], 'difficulty': row['difficulty'], 'score': row['score'], 'duration': row['duration'], 'efficiency': row['efficiency'], 'message': chat['text'], 'speaker_nickname': chat['playerId'], 'timestamp': chat['at']})
        except TypeError: # no one said anything, so write a blank row
            BLANK_SPEAKER = "NULL_SPEAKER"
            BLANK_TIME = "NULL_TIME"
            writer.writerow({'conversation_num': row['id'], 'batch_num': row['batch_num'], 'vis_img': row['vis_img'], 'int_verb': row['int_verb'], 'ort_img': row['ort_img'], 'rep_man': row['rep_man'], 'soc_pers': row['soc_pers'], 'team_size': row['team_size'], 'difficulty': row['difficulty'], 'score': row['score'], 'duration': row['duration'], 'efficiency': row['efficiency'], 'message': "", 'speaker_nickname': BLANK_SPEAKER, "timestamp": BLANK_TIME})


In [7]:
dat_data_csv.head()

,id,batch_num,vis_img,int_verb,ort_img,rep_man,soc_pers,team_size,score,time,eff,chat
0,ws9E8sedkKAbhM2SG,1,High,High,High,High,High,7,88.868950,387.0,0.229636,"{""text"":""I think we can all just start typing ..."
1,SbqSAwmXavcFYKv7S,1,High,High,High,High,High,7,82.831433,354.0,0.233987,"{""text"":""Fungus and tree seem too related."",""p..."
2,SLZGqyCMXq5z3LFQ9,1,High,High,High,High,High,7,83.608805,420.0,0.199069,"{""text"":""So we need words that are as differen..."
3,QGC5hmv4rDf7moy9R,1,High,High,High,High,High,7,83.716433,420.0,0.199325,"{""text"":""if everyone is satisfied toggle the s..."
4,b4ZK7xMAgrTmkCNpx,1,High,Mixed,High,High,High,7,83.276491,236.0,0.352866,"{""text"":""I think list is fine guys"",""playerId""..."


In [8]:
# write the CSOP II data
with open('../raw_data/DAT_conversations_withblanks.csv', 'w', newline='') as csvfile:
    # start writing the header of the CSV
    fieldnames = ["conversation_num", "batch_num","vis_img", "int_verb", "ort_img", "rep_man", "soc_pers", "team_size", "score", "duration", "efficiency", "speaker_nickname", "message", "timestamp"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for index, row in dat_data_csv.iterrows():
        try:
            chat = "[" + row['chat'] + "]" # convert messages to a list of dict objects
            chat_list = eval(chat)
            # filter such that we only look at people who actually said something
            for chat in chat_list:
                writer.writerow({'conversation_num': row['id'], 'batch_num': row['batch_num'], 'vis_img': row['vis_img'], 'int_verb': row['int_verb'], 'ort_img': row['ort_img'], 'rep_man': row['rep_man'], 'soc_pers': row['soc_pers'], 'team_size': row['team_size'], 'score': row['score'], 'duration': row['time'], 'efficiency': row['eff'], 'message': chat['text'], 'speaker_nickname': chat['playerId'], 'timestamp': chat['at']})
        except TypeError: # no one said anything, so write a blank row
            BLANK_SPEAKER = "NULL_SPEAKER"
            BLANK_TIME = "NULL_TIME"
            writer.writerow({'conversation_num': row['id'], 'batch_num': row['batch_num'], 'vis_img': row['vis_img'], 'int_verb': row['int_verb'], 'ort_img': row['ort_img'], 'rep_man': row['rep_man'], 'soc_pers': row['soc_pers'], 'team_size': row['team_size'], 'score': row['score'], 'duration': row['time'], 'efficiency': row['eff'], 'message': "", 'speaker_nickname': BLANK_SPEAKER, "timestamp": BLANK_TIME})
